تمرین چهارم

In [1]:
# Install required packages

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]
!pip install pandas

     |████████████████████████████████| 4.4 MB 16.7 MB/s 
     |████████████████████████████████| 6.6 MB 42.7 MB/s 
     |████████████████████████████████| 596 kB 45.4 MB/s 
     |████████████████████████████████| 101 kB 10.0 MB/s 
     |████████████████████████████████| 316 kB 32.0 MB/s 
     |████████████████████████████████| 1.4 MB 46.4 MB/s 
     |████████████████████████████████| 233 kB 68.1 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 175 kB 52.4 MB/s 
     |████████████████████████████████| 235 kB 58.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
import json
import tqdm
import torch
import re
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

# import plotly.express as px
# import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

from transformers import BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random

In [3]:
def read_data(path, state=None):
    data = json.load(open(path))
    if state is not None:
        data = data.get(state)
    return data

In [4]:
path = "./dataset_annotated_title_fluent.json"
# train_data = read_data(path, "train")

In [5]:
def extract_title(text):
    regex = re.compile("\\n.*\\n")
    title = re.split(regex, text)[0]
    return title


In [6]:
def rewrite_data(path):
    data = read_data(path)
    for key, value in data.items():
        with open(f"{key}.json", "w", encoding="utf-8") as save_file:
            new_result = list()
            for data_item in value:
                data_item["text"] = extract_title(data_item.get("text"))
                new_result.append(data_item)
            final_res = {key: new_result}
            # print(final_res)
            json.dump(final_res, save_file, ensure_ascii=False)
            del new_result
                

In [7]:
rewrite_data(path)

FileNotFoundError: ignored

In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
file_path = "/content/gdrive/MyDrive/NLP/hw4_data/train_clean.csv"

Mounted at /content/gdrive


In [18]:
# train_data = read_data(file_path)
data = pd.read_csv(file_path)
data.head()
data = data.dropna()

In [28]:
data.head()

,Unnamed: 0,text,annotations,normal_prob,interesting_prob,clean,tokens
0,0.0,آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نی...,"['بیان عادی', 'بیان جذاب', 'بیان عادی']",0.666667,0.33333333333333337,آتش سوزی کارخانه جمیل نخ گسترده است اعزام نیرو...,"['آ', '##تش', 'سوزی', 'کارخانه', 'جمیل', 'نخ',..."
1,1.0,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ,"['تجهیزات', 'باکیفیت', 'صنعتی', 'آ', '##ش', '#..."
2,2.0,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری,"['برگزاری', 'جشنواره', 'رسانه', 'ابوذر', 'در',..."
3,3.0,افزایش 80 درصدی فروش سلاح در آمریکا در ژانویه ...,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,افزایش درصدی فروش سلاح در آمریکا در ژانویه,"['افزایش', 'درصدی', 'فروش', 'سلاح', 'در', 'آ',..."
4,4.0,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"['بیان عادی', 'بیان عادی']",1.000000,0.0,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"['بازداشت', 'دو', 'تروریست', 'که', 'در', 'بمب'..."


In [30]:
data["label"] = data.interesting_prob.apply(lambda x: round(float(x)))

In [31]:
data.head()

,Unnamed: 0,text,annotations,normal_prob,interesting_prob,clean,tokens,label
0,0.0,آتش سوزی کارخانه جمیل نخ گسترده است / اعزام نی...,"['بیان عادی', 'بیان جذاب', 'بیان عادی']",0.666667,0.33333333333333337,آتش سوزی کارخانه جمیل نخ گسترده است اعزام نیرو...,"['آ', '##تش', 'سوزی', 'کارخانه', 'جمیل', 'نخ',...",0
1,1.0,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,تجهیزات باکیفیت صنعتی آشپزخانه و کافی شاپ,"['تجهیزات', 'باکیفیت', 'صنعتی', 'آ', '##ش', '#...",0
2,2.0,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,برگزاری جشنواره رسانه ابوذر در دی ماه سال جاری,"['برگزاری', 'جشنواره', 'رسانه', 'ابوذر', 'در',...",0
3,3.0,افزایش 80 درصدی فروش سلاح در آمریکا در ژانویه ...,"['بیان عادی', 'بیان عادی', 'بیان عادی']",1.000000,0.0,افزایش درصدی فروش سلاح در آمریکا در ژانویه,"['افزایش', 'درصدی', 'فروش', 'سلاح', 'در', 'آ',...",0
4,4.0,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"['بیان عادی', 'بیان عادی']",1.000000,0.0,بازداشت دو تروریست که در بمب گذاری مسیر زائران...,"['بازداشت', 'دو', 'تروریست', 'که', 'در', 'بمب'...",0


In [32]:
text = data.text.values
labels = data.label.values

In [22]:
labels = labels.astype(np.float)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [11]:
data = data.explode("annotations")
annots = data.groupby(["annotations"]).count()
labels = annots.index.values.tolist()
print(labels)

['1.0', "['بیان جذاب', 'بیان جذاب', 'بیان جذاب']", "['بیان جذاب', 'بیان جذاب', 'بیان عادی']", "['بیان جذاب', 'بیان جذاب']", "['بیان جذاب', 'بیان عادی', 'بیان جذاب']", "['بیان جذاب', 'بیان عادی', 'بیان عادی']", "['بیان جذاب', 'بیان عادی']", "['بیان جذاب']", "['بیان عادی', 'بیان جذاب', 'بیان جذاب']", "['بیان عادی', 'بیان جذاب', 'بیان عادی']", "['بیان عادی', 'بیان جذاب']", "['بیان عادی', 'بیان عادی', 'بیان جذاب']", "['بیان عادی', 'بیان عادی', 'بیان عادی']", "['بیان عادی', 'بیان عادی']", "['بیان عادی']"]


In [ ]:
data["label"] = data["annotations"]
data = data.dropna(subset=["annotations"])
data["label_id"] = data["label"].apply(lambda x: labels.index(x))
data.head()

In [33]:
tokenizer = BertTokenizer.from_pretrained(
    'HooshvareLab/bert-fa-base-uncased',
    do_lower_case = True
    )
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [34]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [35]:
# Load the BertForSequenceClassification model
model = BertForSequenceClassification.from_pretrained(
    'HooshvareLab/bert-fa-base-uncased',
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [36]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(token_id[train_idx], 
                          attention_masks[train_idx], 
                          labels[train_idx])

val_set = TensorDataset(token_id[val_idx], 
                        attention_masks[val_idx], 
                        labels[val_idx])

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 6

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)
    print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  17%|█▋        | 1/6 [00:11<00:56, 11.28s/it]


	 - Train loss: 0.0819
	 - Validation Accuracy: 0.8158
	 - Validation Precision: 0.4784
	 - Validation Recall: 0.5119
	 - Validation Specificity: 0.8843



Epoch:  33%|███▎      | 2/6 [00:22<00:44, 11.10s/it]


	 - Train loss: 0.0305
	 - Validation Accuracy: 0.7993
	 - Validation Precision: 0.4551
	 - Validation Recall: 0.1955
	 - Validation Specificity: 0.9336

